In [1]:
from langchain.vectorstores.chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings, HuggingFaceBgeEmbeddings
from langchain_core.documents import Document
from unstructured.partition.text import partition_text
from unstructured.chunking.basic import chunk_elements
from unstructured.partition.docx import partition_docx

import time, os, shutil


In [2]:
# CHROMA_PATH = "final_test/unstructured-multi"

# def load_embedding_model(model_path : str):
#     start_time = time.time()
#     encode_kwargs = {"normalize_embeddings": True}
#     local_embedding = HuggingFaceEmbeddings(
#         model_name=model_path,
#         cache_folder="./models",
#         encode_kwargs=encode_kwargs
#     )
#     end_time = time.time()
#     print(f'model load time {round(end_time - start_time, 0)} second')
#     return local_embedding

# embedding = load_embedding_model(model_path="intfloat/multilingual-e5-large")

d:\Python\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


model load time 13.0 second


In [56]:
CHROMA_PATH = "final_test/unstructured-bge"

def load_embedding_model(model_path : str):
    start_time = time.time()
    encode_kwargs = {"normalize_embeddings": True}
    local_embedding = HuggingFaceBgeEmbeddings(
        model_name=model_path,
        cache_folder="./models",
        encode_kwargs=encode_kwargs
    )
    end_time = time.time()
    print(f'model load time {round(end_time - start_time, 0)} second')
    return local_embedding

embedding = load_embedding_model(model_path="BAAI/bge-m3")

model load time 83.0 second


In [57]:
def save_to_chroma(chunks: list[Document]):
    print("Starting Embedding")
    # Clear out the database first.
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)

    # Create a new DB from the documents.
    start_time = time.time()
    db = Chroma.from_documents(
        chunks, 
        embedding, 
        persist_directory=CHROMA_PATH,
        collection_metadata={"hnsw:space": "cosine"}
    )
    db.persist()
    end_time = time.time()
    print(f'embedding time {round(end_time - start_time, 0)} second')
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

# Unstructured

## PNPK 2023

In [58]:
elements = partition_text(filename="final/PNPK 2023 GGK_clean.txt")

In [59]:
chunks = chunk_elements(elements, 
                        max_characters=1800, 
                        new_after_n_chars=1500, 
                        overlap=300, 
                        overlap_all=True)

In [60]:
document_chunks = []
for chunk in chunks:
    temp_metadata = chunk.metadata.to_dict()
    temp_metadata["languages"] = 'ind'
    temp = Document(
            page_content=chunk.text,
            metadata=temp_metadata,
        )
    document_chunks.append(temp)

In [61]:
# unstructured
db = Chroma.from_documents(
        document_chunks, 
        embedding, 
        persist_directory=CHROMA_PATH, 
        collection_metadata={"hnsw:space": "cosine"}
    )
db.persist()

### Tabel PNPK

In [62]:
elements = partition_docx(filename="docx/PNPK tabel.docx")

In [63]:
import re

text = ""
for elem in elements:
    text += elem.text
# print(text)
table_list = re.split(r"Tabel \d+", text)
table_list.pop(0)

''

In [64]:
document_chunks = []
for table in table_list:
    temp = Document(
            page_content=table,
            metadata={'filename':'PNPK 2023 GGK.pdf'}
        )
    document_chunks.append(temp)

In [65]:
db.add_documents(document_chunks)

['78af2ce2-df42-42d8-9e8b-b39fcd030cd1',
 '00cf4f53-f0de-40be-b51b-d8f173066fac',
 'd782baa2-a78b-4568-adc8-b777afabc365',
 'a451738d-d9d2-4632-9d0c-c7ee81e2a5e2',
 '21bc892e-25aa-4718-97f8-96bb92ecfc75',
 '14cdf428-36dc-4da5-a1a6-5a4cdf176285',
 '93e5b540-d977-4aba-be15-3a0b16026801',
 '4f092483-4e90-4d87-9efc-918ab9f4d34a',
 'ab41ac18-5936-473a-be98-da9f679d92a5',
 '8d17b118-a691-4c2c-a296-41aa7e0bfb19',
 '113587e9-1529-4849-8c0b-c5f27849393a',
 'bd6fdad6-bd5e-48e3-b636-caff6e52118c',
 '548a107c-deef-494e-941f-3e2fa701f974',
 '63e68607-11a4-400a-a4d0-880b2d44ac91',
 '9022c3da-65c9-4198-9480-d67a3f29be2b',
 '34f63502-24a9-4110-8498-942928326c1b',
 '7d79cf9b-daf9-42a6-8888-5b7e2ec49b4b',
 'c4212c40-d7b9-4e75-b7d9-79a637a0e020',
 '471e7156-db6d-4325-8455-e8a1e9113694',
 '91c81a42-dcc7-4064-8c85-b501186f6ac7',
 '228e9777-ee09-4dea-b10a-0a7e4e047480',
 'a0afb3bd-0d4d-43b7-a8ed-b7def6836b51',
 'bca78471-db74-4175-8b85-41e889bd4fd6',
 '377c4cde-075a-4b0a-9dff-1821be870759',
 '0e5eb974-d549-

## Nutrisi pada anak dengan penyakit ginjal

In [66]:
elements = partition_text(filename="final/nutrisi_pada_anak_dengan_penyakit_ginjal - Copy_poster.txt")

In [67]:
chunks = chunk_elements(elements, 
                        max_characters=1500, 
                        new_after_n_chars=1000, 
                        overlap=300, 
                        overlap_all=True)

In [68]:
document_chunks = []
for chunk in chunks:
    temp_metadata = chunk.metadata.to_dict()
    temp_metadata["languages"] = 'ind'
    temp = Document(
            page_content=chunk.text,
            metadata=temp_metadata,
        )
    document_chunks.append(temp)

In [69]:
# unstructured
db.add_documents(document_chunks)

['f4710db2-7783-4261-946c-4972e11925c8',
 '168484d8-2e5b-4e2e-b5cd-fc9f63b5c348',
 '5f8b9b48-a63e-408a-993c-523213227366',
 '04684b0b-7dd6-4a90-b04e-f33ede486a27',
 'b368e97b-4fc2-4fca-b6dd-38885dc4b92f',
 '855b15c4-b5e8-48a3-8bff-618f7c035990',
 '80215e08-53b2-40e7-869c-c449a0005309',
 'dbc06895-4c12-4b90-b417-248d91b8e9c1',
 'f00b8fe2-c154-41d4-831d-4f3b6634f8f8',
 '5ff77974-7927-477b-91ff-47b1c4dfff41',
 '19d9fc2c-6adf-46b7-8585-f6b08c60db1a',
 'b6362db3-d8d6-4fdd-9ec0-f691d996cb73',
 '3650f4fe-52ca-4b45-baee-11c94041af19',
 'ccfcb001-cc63-4a8f-9f0f-3d7b1c0cfa1a',
 '5b5828dd-9bfc-428f-aa2d-30de0c730219',
 '5d607467-681d-4071-bc4c-b7eb61aa4969',
 '58b33dca-e2e8-4d8c-ae53-4dff21c07be7',
 '393567c8-9ca9-4ab5-a66c-e42bf913d676',
 'af8b76a5-88dd-4241-90af-e1de6b552846',
 '34e934fa-bdef-4d60-ad87-29126afd1239',
 '07c1860b-f52e-4d9c-8787-bbe3ae567f79',
 '5fbd844a-f258-43d8-99c0-a14677770d04',
 '7495a645-1c10-4fbf-a242-a50b4c5656c2',
 '74c1730e-bd6f-4519-b887-01c59e1adbad',
 '98be840b-5f53-

### Tabel Nutrisi pada Anak dengan Penyakit Ginjal

In [70]:
elements = partition_docx(filename="docx/nutrisi tabel.docx")

In [71]:
import re

text = ""
for elem in elements:
    text += elem.text
# print(text)
table_list = re.split(r"Tabel \d", text)
table_list.pop(0)

''

In [72]:
from langchain_core.documents import Document

document_chunks = []
for table in table_list:
    temp = Document(
            page_content=table,
            metadata={'filename':'nutrisi_pada_anak_dengan_penyakit_ginjal.pdf'}
        )
    document_chunks.append(temp)

In [73]:
db.add_documents(document_chunks)

['112df2a4-c78d-42d3-bf76-ced20f1e6fca',
 '64f41d44-8051-4416-9286-d62f287730a7',
 '6545ecdc-9a44-4850-8d5d-44fe9cbc022d',
 '4e23d188-1db7-49b3-98c2-fd8a7e21c03c',
 '314d3d8a-805d-4188-962f-d91e4e9d8d5a',
 'f069f642-c155-46f6-bc90-c53fc9f30da8',
 'ce707ad4-084a-473b-bdca-e8c7a1e7b525',
 '70a52d5e-2b38-4471-b7d4-176830cde4a3']

## Tatalaksana Hemodialisis pada Anak dan Bayi

In [74]:
elements = partition_text(filename="final/Tatalaksana_Hemodialisis_pada_Anak_dan_Bayi_clean.txt")

In [75]:
chunks = chunk_elements(elements, 
                        max_characters=1500, 
                        new_after_n_chars=1000, 
                        overlap=300, 
                        overlap_all=True)

In [76]:
document_chunks = []
for chunk in chunks:
    temp_metadata = chunk.metadata.to_dict()
    temp_metadata["languages"] = 'ind'
    temp = Document(
            page_content=chunk.text,
            metadata=temp_metadata,
        )
    document_chunks.append(temp)

In [77]:
db.add_documents(document_chunks)

['8211a60f-c425-42cb-8ba5-9f4a589e89ea',
 'b3fa3761-612a-4134-9ed2-42a81cb40329',
 '1d607717-fc09-45c1-946e-8d633077ca41',
 '830831d0-0ec6-48c5-9f08-a072188dee4e',
 'c5fbc31b-bdf4-4864-b585-0c9ab3914b32',
 '428b376a-469d-4ec2-98eb-95af24da080e',
 '7f25921d-b14d-40ec-bcb5-3c947c67c59f',
 '5ab03e45-c6e3-4edd-9d20-cd55c91227ba',
 '03fecae8-2b5b-41ee-bf47-4ba0465798bf',
 '8e338029-1721-4bc5-bfab-37a5a1d50d02',
 '03554512-22ba-4f7e-82f5-51dc5da337fc',
 '6cb07b20-2bf9-414b-bb36-1b7d0184a3f6',
 '0d101d98-ede7-406a-987a-41989b25c4a7',
 'b5fc5467-115e-4dac-8540-644a9c2108d3',
 'be4bf580-432d-4e23-825c-d5cdd0ac090f',
 'ca39103a-313e-4dc2-900d-7475ba4d0b9a',
 'c3376348-db65-433a-ba3e-9fbafef14b3a',
 '5494791a-4165-4c97-b33d-cb1dee790dfc',
 '5766997a-273e-4c69-a518-966656c4cfe3',
 '0324d6e1-aeb7-4d9c-ae1b-a343ce864f01',
 '6b9ee69c-0ffb-4b2e-b854-526795a0012c',
 'b0261333-5190-4ef5-8b15-66c0c3ff4b2a',
 '2afb4f33-e52a-4df9-b99c-0984fed0f4d9',
 '9ca16966-38bf-412c-8b4b-247eb3c2a2a1',
 '2b530be2-28f5-

## Tatalaksana Penyakit Ginjal Kronik pada Anak

In [78]:
elements = partition_text(filename="final/Tatalaksana_Penyakit_Ginjal_Kronik_pada_Anak_clean_split_final.txt")

In [79]:
chunks = chunk_elements(elements, 
                        max_characters=1500, 
                        new_after_n_chars=1000, 
                        overlap=300, 
                        overlap_all=True)

In [80]:
document_chunks = []
for chunk in chunks:
    temp_metadata = chunk.metadata.to_dict()
    temp_metadata["languages"] = 'ind'
    temp = Document(
            page_content=chunk.text,
            metadata=temp_metadata,
        )
    document_chunks.append(temp)

In [81]:
db.add_documents(document_chunks)

['2cf3e20d-5070-4950-80fb-f20aef1eefd3',
 'b4151899-9818-4c58-8db1-052a6f0e40e7',
 '3a56636d-2cfa-485e-b144-e4d5e99b2059',
 'c923e61e-f1e7-4c1d-b941-a8d7994899d5',
 '4d06bb29-42c2-4369-8456-9af99d264588',
 '5427ca11-3f5e-40b7-be30-e3a7f5341980',
 'fcb7060a-293b-4c88-87a2-799e88444303',
 '00200503-f780-4ee8-b1b7-c103587b14a2',
 '8612d95a-091e-4a55-9b66-12e78781b03c',
 '5adfedba-60c9-42f7-9960-e101528b7add',
 '41c2b7a4-c53e-4884-964b-3edab9c0806d',
 '731e32f2-2cc9-4de3-abce-d22ab67594ea',
 'c4aa5d91-9078-47e8-8ea8-750a18f58c70',
 '98582fc9-3ef7-49ec-9e60-b56359ca5076',
 '361c9c32-3757-48eb-a2df-d94c9f652082',
 '016d6028-566a-46e2-9ddd-f7f7e6e8d488',
 '5ff68dd0-bb94-4072-8bd3-1261a18781bd']

### Tabel Tatalaksana Penyakit Ginjal Kronik pada Anak

In [82]:
tata_laksana_tabel_elements = partition_docx(filename="docx/tatalaksana anak tabel.docx")

In [83]:
import re

tata_anak_tabel_text = ""
for elem in tata_laksana_tabel_elements:
    tata_anak_tabel_text += elem.text
# print(tata_anak_tabel_text)
table_list = re.split(r"Tabel \d", tata_anak_tabel_text)
table_list.pop(0)

''

In [84]:
document_chunks = []
for table in table_list:
    temp = Document(
            page_content=table,
            metadata={'filename':'Tatalaksana_Penyakit_Ginjal_Kronik_pada_Anak.pdf'}
        )
    document_chunks.append(temp)

In [85]:
db.add_documents(document_chunks)

['b924465f-c8eb-4733-857e-7d24e02ca88c',
 '8208ceaa-0002-4730-ad4b-b8a86e1adb4e',
 '94515486-c494-49d4-833b-b445ef79921a',
 'ed5c98a5-be0a-493e-a419-30976fbd32d0',
 'b150eccc-3969-4d4f-8cfc-713ace636c10']


# Langchain

In [86]:
from langchain_community.document_loaders import TextLoader
# CHROMA_PATH = "final_test/langchain-multi"
CHROMA_PATH = "final_test/langchain-bge"

In [87]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_text(documents: list[Document]):
    print("Starting chunking")

    "Recursive Splitter"
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=300,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")
    print(f"len docs {len(chunks)}")
    document = chunks[10]
    print("page content \n", document.page_content)
    print("doc metadata \n", document.metadata)
    return chunks

## PNPK 2023

In [88]:
loader = TextLoader("final/PNPK 2023 GGK_clean.txt", encoding="UTF-8")
documents = loader.load()

In [89]:
# langchain splitter
chunks = split_text(documents)
db = Chroma.from_documents(
        chunks, 
        embedding, 
        persist_directory=CHROMA_PATH, 
        collection_metadata={"hnsw:space": "cosine"}
    )
db.persist()

Starting chunking
Split 1 documents into 377 chunks.
len docs 377
page content 
 albuminuria atau ACR urin.  
3. Pasien dengan Hematuria
Pasien dengan hematuria dilakukan pemeriksaan: [Peringkat bukti:
Level IV]
a.Pasien dengan hematuria harus disingkirkan kemungkinanpenyebab lain seperti infeksi saluran kemih
b.Pasien dengan hematuria ma
kroskopik/mikroskopik persisten,
tanpa adanya proteinuria dan penurunan eGFR dirujuk kespesialis urologi untuk pemeriksaan lebih lanjut
c.Pasien dengan hematuria makroskopik/mikroskopik, denganpenyerta klinis hipertensi, proteinuria, dan penurunan eGFR
doc metadata 
 {'source': 'final/PNPK 2023 GGK_clean.txt', 'start_index': 8941}


### Tabel PNPK

In [90]:
elements = partition_docx(filename="docx/PNPK tabel.docx")

In [91]:
import re

text = ""
for elem in elements:
    text += elem.text
# print(text)
table_list = re.split(r"Tabel \d+", text)
table_list.pop(0)

''

In [92]:
document_chunks = []
for table in table_list:
    temp = Document(
            page_content=table,
            metadata={'filename':'PNPK 2023 GGK.pdf'}
        )
    document_chunks.append(temp)

In [93]:
db.add_documents(document_chunks)

['11d99ee1-3dbc-4e7e-917c-2cbbaafc3ace',
 '9ecde06b-60ca-4d7c-930b-c6dad3139f4c',
 'e4a0841a-1a0d-41ef-be0d-2ed55e41ac4a',
 '8ca81ec1-625b-48ef-96dc-d18837c521e0',
 '8642c19f-ae4c-46b0-917e-45f792430a5f',
 'd43d6b42-3651-4c2f-b947-79fe9209a2a2',
 '3eff185d-4d46-403b-b837-f5199cd11480',
 'c5a709d5-b182-44be-b2e2-f9625923c7d1',
 '2bca7ca3-f197-49d7-9d2d-32d9fac63745',
 'cf94d203-6a9f-41c5-99b7-5357175ef22c',
 '0d6c82fb-2e2d-43fa-ac6d-44857c3dcaed',
 '6dfe652b-0850-410c-a155-193eaab4653c',
 '36d52e35-0890-4837-ae05-81937862f240',
 '75b0d59c-7289-4b69-8127-5aebef3bdb51',
 '4094c80f-5275-4a00-b220-91ec1290da8d',
 'a987215f-13a4-4917-b1e0-32c624e60f27',
 '7fd05219-c996-4386-8869-385ca137505e',
 '3d5840bd-76fd-479e-98ee-8b3b58b20946',
 '414f589f-1c4c-4624-9d9d-6e0a5ca345bd',
 '57d2997e-278d-4779-a90f-fc2698e53e8d',
 'd9a620c0-5bed-444e-9339-6ec4f6db5e98',
 '15e8c76b-1373-49ee-882c-8725c391341b',
 'b960bed9-5bb9-427d-bf60-9ba86537b414',
 '98ec604d-81bf-424c-bde3-6268fe9a9fd9',
 '34b3f79e-6397-

## Nutrisi pada anak dengan penyakit ginjal

In [94]:
loader = TextLoader("final/nutrisi_pada_anak_dengan_penyakit_ginjal - Copy_poster.txt", encoding="UTF-8")
documents = loader.load()

In [95]:
# langchain splitter
chunks = split_text(documents)
db.add_documents(chunks)

Starting chunking
Split 1 documents into 20 chunks.
len docs 20
page content 
 d. Kekurangan berat badan lebih dari 10% berat badan ideal. e. Indeks massa tubuh kurang dari persentil 5 atau lebih dari persentil 85 untuk tinggi badan menurut umur. f. Kenaikan berat badan yang tidak adekuat dan perawakan pendek. g. Abnormalitas nutrisi yang berhubungan dengan gangguan biokimiawi. 4. Terapi kebutuhan energi Penyebab gangguan asupan kalori pada pasien PGK adalah penurunan selera makan dan muntah. Angka kematian PGK lebih tinggi pada anak dengan indeks massa tubuh di bawah atau di atas nilai normal. Faktor penyebab gangguan selera makan pada pasien PGK antara lain:  a. Pada anak dengan poliuria, rasa haus lebih  nyata dibanding rasa lapar . b. Pasien lebih menyukai makanan bergaram  dibandingkan makanan manis sebagai asupan  energi. c. Akumulasi selera makan yang meregulasi sitokin dan hormon. d. Refl  uks gastroesofagus. e. Gangguan motilitas lambung. f.  Pengosongan lambung terlambat. g. 

['c4a3f705-76f7-40f9-b05c-983c33ee1937',
 '0e3c0643-7c1e-41f5-9d09-8ec933fdc661',
 'e2296d33-8afe-47b4-bb7d-3a2aaffa0662',
 '0e3481fd-542d-4622-a3d4-bf109de172e2',
 '6898a510-5185-4c72-8360-0031dafef709',
 '62161fa2-daec-4390-bfd1-06be8d7db799',
 'ed57fe78-9d79-4bcd-a8cf-05bab3193bd0',
 '53f5106e-c99f-436d-ae70-39ace80662bc',
 'cfebbfb4-10a3-449c-9fc5-593849ae9d19',
 '86defd21-dfd6-4ee2-9062-c41b1e868c93',
 'a9165060-5853-4955-bd0d-3f3a8378f9ca',
 'cdd90398-af2c-46eb-96be-648176855284',
 '6fbf9b22-2b63-4fa0-9c44-71f3388a7636',
 '715f322e-7604-4b6e-9797-c1946112919e',
 'f62e0c01-c120-45c7-b2b2-060018e6d216',
 'ec880734-4a5c-460b-a119-f018c3d15584',
 'b22628b9-8afb-4677-b6a3-333c1be922fb',
 '90d583bc-9458-4a16-83fd-c39668c84def',
 'd318eb56-d31c-4b1b-8ca5-93b6781b6a1d',
 '7a108d9f-6d32-4b94-8a99-c8b54f0717bd']

### Tabel Nutrisi pada Anak dengan Penyakit Ginjal

In [96]:
elements = partition_docx(filename="docx/nutrisi tabel.docx")

In [97]:
import re

text = ""
for elem in elements:
    text += elem.text
# print(text)
table_list = re.split(r"Tabel \d", text)
table_list.pop(0)

''

In [98]:
for table in table_list:
    print(table)

Klasifikasi derajat penyakit ginjal kronik.Derajat 1: Kerusakan ginjal dengan LFG normal atau meningkat, GFR ≥90 mL/min/1.73 m².Derajat 2: Kerusakan ginjal dengan penurunan ringan LFG, GFR 60-89 mL/min/1.73 m².Derajat 3: Penurunan sedang LFG, GFR 30-59 mL/min/1.73 m².Derajat 4: Penurunan berat LFG,  GFR 15-29 mL/min/1.73 m².Derajat 5: gagal ginjal,  GFR <15 mL/min/1.73 m²atau dialisis.
 Rekomendasi parameter dan frekuensi penilaian nutrisi untuk anak dengan PGK derajat 2 sampai 5 dan 5-dialisis (5D)Umur < 1 tahun, PGK 2 – 3, Asupan makanan : 0,5 – 3 bulan, Persentil tinggi badan terhadap umur : 0,5 – 1,5 bulan, Persentil velocity tinggi badan terhadap umur : 0,5 – 2 bulan, Perkiraan berat kering dan persentil berat badan terhadap umur : 0,5 – 1,5 bulan, Indeks massa tubuh : 0,5 – 1,5 bulan, Persentil lingkar kepala : 0,5 – 1,5 bulanUmur < 1 tahun, PGK 4 – 5, Asupan makanan : 0,5 – 3 bulan, Persentil tinggi badan terhadap umur : 0,5 – 1,5 bulan, Persentil velocity tinggi badan terhadap 

In [99]:
from langchain_core.documents import Document

document_chunks = []
for table in table_list:
    temp = Document(
            page_content=table,
            metadata={'filename':'nutrisi_pada_anak_dengan_penyakit_ginjal.pdf'}
        )
    document_chunks.append(temp)

In [100]:
db.add_documents(document_chunks)

['4c4b909b-156a-4bec-96b3-8ab517b45a18',
 '065e3554-71a0-4bef-8801-c6e6c885ba80',
 '37034ea8-8701-4541-b9e6-df2053cc448e',
 'e0e1cb4a-a85d-4410-97e1-649580583d9e',
 'de44379a-90ad-4d32-92e6-0455fa34382d',
 '0e60f01a-a61f-4062-a9a8-d6a88d4d91a3',
 '7f0e901d-4e02-4b4d-aa36-f154c28ebd7b',
 '8dfcae61-d2e6-4aee-a76f-da1ec9562106']

## Tatalaksana Hemodialisis pada Anak dan Bayi

In [101]:
loader = TextLoader("final/Tatalaksana_Hemodialisis_pada_Anak_dan_Bayi_clean.txt", encoding='utf-8')
documents = loader.load()

In [102]:
chunks = split_text(documents)
db.add_documents(chunks)

Starting chunking
Split 1 documents into 27 chunks.
len docs 27
page content 
 mendasar antara  hemodialisis dewasa dan pediatrik. Misalnya,  pada pediatrik, aliran darah dan hemodialyzer  dipilih atas dasar bahwa anak-anak dapat  mentolerir 8% (maksimum 10%) total volume  darah mereka di sirkuit ekstrakorporeal  berdasarkan perkiraan volume darah total 80  mL/ kg untuk bayi dan 70 mL/ kg untuk anak  yang lebih tua. Jika sirkuit hemodialisis terkecil  yang tersedia melebihi volume kritis ini, dapat  ditutupi dengan larutan albumin manusia  4,5% atau darah donor untuk mencegah  gejala hipovolemia. Karena aliran darah bayi dan anak memiliki  kaliber lebih kecil, kecepatan pompa darah  cenderung lebih cepat, dengan sasaran 8–10  mL/kg/menit, dibandingkan 3-5 mL/kg/menit  untuk hemodialisis dewasa. Penggulung  kepala pompa darah (blood pump head  rollers) harus disesuaikan dengan ukuran  aliran dialisis, hemolisis mekanik dapat terjadi  jika aliran darah dewasa digunakan dalam  mesin diali

['351cac43-94db-4a66-abc7-e46cbfb36520',
 '60a4e4ce-de79-4d7e-8920-cc0e9280711f',
 '29a8bcf6-a108-4604-b030-d4f23845988b',
 'de0c317a-4590-4ba3-a70e-93d83632c977',
 '1c26b061-04a4-4364-b2c3-e415ef8dfe3a',
 '3e378cc6-99a7-435e-9fc8-57e358712b1d',
 '0471d3ff-9d6b-4e12-a28a-aead47d91338',
 '36936719-68e9-407d-ab9b-9b1199978988',
 '9b844f19-23da-4ff2-be3a-b109bcba9709',
 '3a225c8a-a5e7-49e2-ba4b-8d1aa48dfb49',
 '28dde195-79c3-4b9d-a35f-e04717b40599',
 '04151298-4cc0-4817-891d-7c4712eded2a',
 '8a47b3ce-9869-4e3a-8151-0f9f54e79c78',
 'de14afcf-cce3-49df-8aae-ae1182359984',
 '0c81bd5f-a412-4c1c-a157-ff58817f10b4',
 'dc781849-2472-43f0-bc17-44823d256960',
 'e91778d8-ad8e-4fbd-91c3-e5f00a2f7bdb',
 '109d5c51-7e58-4a73-84a8-5818b1453bad',
 '299e3908-b2da-47c5-b45b-33a99e5384f5',
 '62c1a35b-d98a-42c5-ac54-f0af2c7484c1',
 '7c9d52be-a146-4670-8430-db4643727b65',
 '3c581f1b-a1ee-49af-9a98-452410af45ec',
 '3c02156c-3c3e-4d6d-8616-c3e47a6b5237',
 'be4ac501-9910-419c-a37f-b73092c03339',
 '9c0bb0ec-2e22-

## Tatalaksana Penyakit Ginjal Kronik pada Anak

In [103]:
loader = TextLoader("final/Tatalaksana_Penyakit_Ginjal_Kronik_pada_Anak_clean_split_final.txt", encoding='utf-8')
documents = loader.load()

In [104]:
chunks = split_text(documents)
db.add_documents(chunks)

Starting chunking
Split 1 documents into 12 chunks.
len docs 12
page content 
 Penyakit ginjal kronis stadium 5 merupakan indikasi untuk transplantasi. Meskipun demikian, tidak semua 
pasien dengan penyakit ginjal kronis stadium 5 dapat 
menjadi kandidat untuk transplantasi ginjal. Prosedur ini 
dapat terlalu berisiko bagi sebagian karena komorbiditas
yang telah diderita pasien atau karena kontraindikasi 
tertentu, seperti infeksi kronis yang akan dieksaserbasi oleh imunosupresi. Kontraindikasi absolut bagi transplantasi ginjal adalah keganasan aktif, terutama jika telah bermetastasis. Infeksi HIV dan potensi rekurensi penyakit ginjal juga perlu dipertimbangkan 
dalam rencana transplantasi. 
Prognosis pasien dengan penyakit ginjal kronis adalah 
bervariasi menurut stadium dan penatalaksanaan yang dilakukan. Dengandeteksi dan penatalaksanaan dini,morbiditas dan mortalitas diharapkan dapat diturunkan.  
3.Kesimpulan
Penyakit ginjal kronis pada anak dapat disebabkan penyakit 
kongenital, 

['63938315-4350-4979-a17c-6320c29217ff',
 '82cc9868-5bc2-43b2-b9b1-73b5fd2639a3',
 '881c4e52-be30-452d-8b33-46dedb027a16',
 '994e9940-a9fa-4b68-96a8-bd992b98b277',
 '4d0c2282-e159-4a55-b382-5ac6d7340857',
 'ab8c7af4-26a8-45de-9896-ae0c9ea4c4e1',
 '378b3aa9-92c5-4a27-bb4f-9ccf9a0f2de2',
 '8357191f-d8ab-4432-876a-43a008b2be3d',
 '4531c78e-8c31-4c07-a47d-6695fcb2fac0',
 'c5aa9c3f-d632-45e2-a33b-6a458d13fcf8',
 '72bed413-2728-4f0f-a2ee-3c22ff50d59b',
 'c3d730a1-f2ec-4360-a180-f3b578d4e00c']

### Tabel Tatalaksana Penyakit Ginjal Kronik pada Anak

In [105]:
tata_laksana_tabel_elements = partition_docx(filename="docx/tatalaksana anak tabel.docx")

In [106]:
import re

tata_anak_tabel_text = ""
for elem in tata_laksana_tabel_elements:
    tata_anak_tabel_text += elem.text
# print(tata_anak_tabel_text)
table_list = re.split(r"Tabel \d", tata_anak_tabel_text)
table_list.pop(0)

''

In [107]:
document_chunks = []
for table in table_list:
    temp = Document(
            page_content=table,
            metadata={'filename':'Tatalaksana_Penyakit_Ginjal_Kronik_pada_Anak.pdf'}
        )
    document_chunks.append(temp)

In [108]:
db.add_documents(document_chunks)

['29a825aa-4db0-45e8-bbda-59fd87b1da26',
 'e969cfe6-18c7-4e64-89c3-12867f226cde',
 '51b90175-dd56-49c5-98d3-fa36448770dd',
 '6868f37b-7c0f-4ea7-b0ca-c0a9a3f0d4f4',
 '8860c201-696e-4b45-8d55-fbbce4061238']